### ДИПЛОМНАЯ РАБОТА

#### ТЕМА: модели генерации текста на примере генератора хайку

#### ПЛАН РАБОТЫ

- Character-based model
- Word-based model
- GAN / Sense-vector? Autoencoders?

#### ВВЕДЕНИЕ

здесь будет текст 

In [2]:
# prepare data

file_path_src = "haiku"
with open(file_path_src, encoding="utf-8") as f:
    lines = f.readlines()
    by3lines = []
    allHaiku = []
    for line in lines:
        if line == "\n":
            allHaiku.append(by3lines)
            by3lines = []
        else:
            by3lines.append(line.lower())
    # print(allHaiku);

In [3]:
def to_oneline(data):
    return ["".join(row) for row in data]

максимальная длина стихотворения

In [4]:
hh = to_oneline(allHaiku)

In [15]:
hh

['луна-проводник \nзовет: "загляни ко мне". \nдом у дороги. \n',
 'скучные дожди, \nсосны разогнали вас. \nпервый снег в лесу. \n',
 'протянул ирис \nлистья к брату своему. \nзеркало реки. \n',
 'снег согнул бамбук, \nсловно мир вокруг него \nперевернулся. \n',
 'парят снежинки \nгустою пеленою. \nзимний орнамент. \n',
 'полевой цветок \nв лучах заката меня \nпленил на миг. \n',
 'вишни расцвели. \nне открыть сегодня мне \nтетрадь с песнями. \n',
 'веселье кругом. \nвишни со склона горы, \nвас не позвали? \n',
 'над вишней в цвету \nспряталась за облака \nскромница луна. \n',
 'тучи пролегли \nмежду друзьями. гуси \nпростились в небе. \n',
 'леса полоса \nна склоне горы, словно \nпояс для меча. \n',
 'все, чего достиг? \nна вершины гор, шляпу \nопустив, прилег. \n',
 'ветер со склонов \nфудзи в город забрать бы, \nкак бесценный дар. \n',
 'долгий путь пройден, \nза далеким облаком. \nсяду отдохнуть. \n',
 'взгляд не отвести - \nлуна над горной грядой, \nродина моя. \n',
 'новогодние \n

In [5]:
max(map(lambda s: len(s), hh))

84

In [18]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation

import numpy as np
import random
import sys
import io
import gensim
import string

In [29]:
s = "asjo,fdjk;dja\nso,oio!kod.kjods;dkps"
# s.translate({",": None, "!": "q"})
s.translate(str.maketrans('\n',' ',string.punctuation))

'asjofdjkdja sooiokodkjodsdkps'

In [ ]:
"".translate

In [34]:
sentences = [[word for word in line.translate(str.maketrans('\n…','  ',string.punctuation)).split()] for line in hh]

In [35]:
sentences

[['лунапроводник', 'зовет', 'загляни', 'ко', 'мне', 'дом', 'у', 'дороги'],
 ['скучные',
  'дожди',
  'сосны',
  'разогнали',
  'вас',
  'первый',
  'снег',
  'в',
  'лесу'],
 ['протянул', 'ирис', 'листья', 'к', 'брату', 'своему', 'зеркало', 'реки'],
 ['снег',
  'согнул',
  'бамбук',
  'словно',
  'мир',
  'вокруг',
  'него',
  'перевернулся'],
 ['парят', 'снежинки', 'густою', 'пеленою', 'зимний', 'орнамент'],
 ['полевой', 'цветок', 'в', 'лучах', 'заката', 'меня', 'пленил', 'на', 'миг'],
 ['вишни',
  'расцвели',
  'не',
  'открыть',
  'сегодня',
  'мне',
  'тетрадь',
  'с',
  'песнями'],
 ['веселье',
  'кругом',
  'вишни',
  'со',
  'склона',
  'горы',
  'вас',
  'не',
  'позвали'],
 ['над',
  'вишней',
  'в',
  'цвету',
  'спряталась',
  'за',
  'облака',
  'скромница',
  'луна'],
 ['тучи', 'пролегли', 'между', 'друзьями', 'гуси', 'простились', 'в', 'небе'],
 ['леса', 'полоса', 'на', 'склоне', 'горы', 'словно', 'пояс', 'для', 'меча'],
 ['все',
  'чего',
  'достиг',
  'на',
  'вершины',

тренируем собственную Word2Vec модель

In [36]:
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=2, iter=500)

In [37]:
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in ['цветок', 'ветер', 'дожди']:
  similar_words = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:3])
  print('  %s -> %s' % (word, similar_words))

Result embedding shape: (1984, 100)
Checking similar words:
  цветок -> ириса (0.83), еще (0.83), сливовый (0.81)
  ветер -> взмах (0.79), весла (0.77), тогда (0.75)
  дожди -> льют (0.91), скучные (0.78), долгие (0.77)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


заготовим функции, преобразующие слова в индексы и обратно

In [38]:
def word2idx(word):
  return word_model.wv.vocab[word].index

def idx2word(idx):
  return word_model.wv.index2word[idx]

узнаем максимально длинное предложение

In [40]:
max_sentence_len = max(map(lambda s: len(s), sentences))
max_sentence_len

15

подготовим данные для модели...

In [41]:
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

train_x shape: (390, 15)
train_y shape: (390,)


Построим архитектуру модели. Входным слоем сделаем Embedding слой

In [42]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [43]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [44]:
def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = [
    'цветок',
    'ветер',
    'дом',
    'долгие',
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))

In [45]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=100,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
390/390 [==============================] - 1s 2ms/step - loss: 7.5949

Generating text after epoch: 0
цветок... -> цветок краб ветрам горы утром срез места ущелья цветенья жду воробей
ветер... -> ветер радугу жены выскочил лесного подул касается хижина твоего коршуна снежный
дом... -> дом хризантемы горсть ум грустно может сушатся ловит поясе замерзшая засверкали
долгие... -> долгие соломы ней жить кленовые груду около осенние шиповник пляшут самозабвенно
Epoch 2/100
390/390 [==============================] - 0s 204us/step - loss: 7.4383

Generating text after epoch: 1
цветок... -> цветок успела кумирни примостилась гостинной сломал санитарная шелестит близится выехал пуху
ветер... -> ветер берега свои воде везде выскочил какойто стремглав дерево нам ком
дом... -> дом скорлупка горной прост украл брести поймал гибкой из самозабвенно дождлив
долгие... -> долгие самой твоя спину облачка выжженным звонко ко стучится поймёте чего


390/390 [==============================] - 0s 261us/step - loss: 5.6431

Generating text after epoch: 17
цветок... -> цветок прежней пиона один ничегошеньки напрасно достоин воды плотно чана чьито
ветер... -> ветер цветы вдруг разогнали гнется ставни спряталась западу вечера вишнями проблеск
дом... -> дом дерево тебе глядя позвали одноцветен первую подножии бесполезно рассвет серой
долгие... -> долгие светлый груди ком пределами ком зажёг пляшут летняя гостя грабитель
Epoch 19/100
390/390 [==============================] - 0s 266us/step - loss: 5.6299

Generating text after epoch: 18
цветок... -> цветок замертво цапли волну вдалеке блестках грозой раскрываются дар стайка дорога
ветер... -> ветер закружит фудзи белая глядят тяжел огни улетает цветы цветенье молнию
дом... -> дом пожелал рыбачьи пламя закрой освободился перевала ворчит стучится улиток летят
долгие... -> долгие тянула друг господ свои хризантеме прежде верно руке зной славное
Epoch 20/100
390/390 [=========================

390/390 [==============================] - 0s 233us/step - loss: 5.2710

Generating text after epoch: 35
цветок... -> цветок гонит уха против вдали тихотихо молитвы скал свежий примостилась глоток
ветер... -> ветер кошка краю треснула само тёмная нектар выпил летом холодных горные
дом... -> дом углах свежем грядой вырос лицах водою дыры хризантема рыжая лука
долгие... -> долгие пустом росинка прекрасные весенний растают ночи голоса влетает нищий небе
Epoch 37/100
390/390 [==============================] - 0s 221us/step - loss: 5.2412

Generating text after epoch: 36
цветок... -> цветок угаснет малы их с заморозки рыбака стихи белеет заря глупей
ветер... -> ветер сама студит красно их креветок своих листву тетрадь упавший рассвете
дом... -> дом гудит треснула часточасто прилип прохладное плен тяжел корням дятелтрудяга а
долгие... -> долгие остров висит деревце рядом просыпайся примостился убивающие песнями хочется силу
Epoch 38/100
390/390 [==============================] - 0s 209us/ste

390/390 [==============================] - 0s 209us/step - loss: 4.7222

Generating text after epoch: 53
цветок... -> цветок холмом мотонобу сотни небеса влечет луной радостью дождлив бочка выше
ветер... -> ветер плывущий сильный пришла ком пожалуй шип слушаю шмель снежный красивы
дом... -> дом дар ней поезд невидимы разлив погребли пьет торопливо теплом земному
долгие... -> долгие незнакомый тушью гнетом повисло ущелья цикада часточасто году берег хрупки
Epoch 55/100
390/390 [==============================] - 0s 211us/step - loss: 4.6875

Generating text after epoch: 54
цветок... -> цветок далеко стрекоза никак тоской прохладный началась» само дворе жду началась»
ветер... -> ветер чуть без с посвоему озарит замешался увяли ирис бочке хризантеме
дом... -> дом базар гвоздики удар моя рассветной дождлив где твое вековой погляди
долгие... -> долгие достоин молнию вечерний тобою плывет облака твои полей полдень берега
Epoch 56/100
390/390 [==============================] - 0s 204us/step - 

390/390 [==============================] - 0s 207us/step - loss: 4.0884

Generating text after epoch: 71
цветок... -> цветок обод пеленою очага ручей былинке пролетный весть бурьяна черен ливни
ветер... -> ветер ладонях воздух дальний белеет успела среди спряталась море дятел монахини
дом... -> дом месяц ветвях брызги пируют месте зимняя ирис вчера детворой посадили
долгие... -> долгие согнул ему леса прохладное подходит прохладен затихла рыбака одноцветен треснула
Epoch 73/100
390/390 [==============================] - 0s 207us/step - loss: 4.0592

Generating text after epoch: 72
цветок... -> цветок вьюнок сильнее старуха мечты спокойный идет выскочил прост воробей людей
ветер... -> ветер реки облачка клочок одна глицинии ракушка вином отдай прозрачная будто
дом... -> дом взор тебе громкий прольется земному полей орнамент летних полдень пальмах
долгие... -> долгие загляни беседа нас ночи винный дороги рукой хризантеме заснул собой
Epoch 74/100
390/390 [==============================] 

390/390 [==============================] - 0s 213us/step - loss: 3.4454

Generating text after epoch: 89
цветок... -> цветок грозовой бредет угаснет грибок сильный порой цветок комары свежем на
ветер... -> ветер хоть движутся ну миг блекли игла ножках путь цветок воды
дом... -> дом подарок цветы белит столицы белеет спряталась ручей когда красно озарились
долгие... -> долгие вьюнка лохмотья небо ловцов смотрит воду верит росинка грузный девушка
Epoch 91/100
390/390 [==============================] - 0s 241us/step - loss: 3.4153

Generating text after epoch: 90
цветок... -> цветок же полям перед мышь улитка залили осталось сам речные снег
ветер... -> ветер пред кажется их утром пугало кустик бабочка ударил ели образ
дом... -> дом твоего которой гостя листья сколько горит весла моста стучит света
долгие... -> долгие офисе летят близятся корням полоса вечерняя дверь стебли студит стало
Epoch 92/100
390/390 [==============================] - 0s 214us/step - loss: 3.3783

Generating text af